In [ ]:
#This notebook is a test. Should give same outputs as productivity, but be shorter as I've *successfully* factored the code into scripts.

In [ ]:
exec(open("data_prep/subnat_data_import.py").read()) 

In [ ]:
from data_prep.subnat_data_clean import *

In [ ]:
#Convert apprenticeships into a rate
app_rate = app_completion.merge(app_start, on="AREACD", how='inner')
app_rate['Value'] = app_rate['Value_x']/app_rate['Value_y']
app_rate['Indicator'] = 'Approx apprenticeship completion rate'
app_rate.pop('Value_x')
app_rate = app_rate[['AREACD', 'Indicator', 'Value']]

In [ ]:
#Select our groups of metrics
productivity = [gva, high_growth]
pay = [weekly_pay, gdi, employ_rate, neet]
lstandards =[male_hle, female_hle, satisfaction, green_space, inactive]
skills = [gcses, app_rate, maths_5, lit_5, comm_5]


In [ ]:
#Define the groups of metrics we have to streamline the processing.
#Do all the cleaning at once.
groups = [productivity, pay, lstandards, skills]
for group in groups:
    group = clean_groups(group)

In [ ]:
#Do the clustering.
from cluster_analysis.cluster_functions import *

In [ ]:
productivity_wide = metrics_to_table(productivity)
pay_wide = metrics_to_table(pay)
lstandards_wide = metrics_to_table(lstandards)
skills_wide = metrics_to_table(skills)

In [ ]:
km_model = make_clustering_model()
map = clusters_and_map(metrics_to_table(productivity), km_model)

In [ ]:
analysis = clusters_and_plots(productivity_wide, km_model)

In [ ]:
#Transform each group into a 'cluster' version using the basic KMeans model.
prod_cluster = clusters_and_plots(productivity_wide, km_model, show_plots=True)
pay_cluster = clusters_and_plots(pay_wide, km_model, show_plots=True)
lstandards_cluster = clusters_and_plots(lstandards_wide, km_model, show_plots=True)
skills_cluster = clusters_and_plots(skills_wide, km_model, show_plots=True)   


In [ ]:
from data_vis.bubble_plots import *
all_clusters = [prod_cluster, pay_cluster, lstandards_cluster, skills_cluster]

#Annoyingly this function only works when declared at main level (not in module due to globals())
def variable_to_name(var):
    """Helper function with takes in a variable and return a string of that variables name.
    If the variable doesn't exist, will return null.
    Might go wrong if there are multiple declared variables with identical values!
    """
    for name in globals():
        if (eval(name) is var) and (not name.startswith("_")): #Omit variables in globals with begin with _, as __ is shorthand for last evaluated result and in jupyter notebooks _1 return output of cell evaluation 1 etc.
            return(name)

labels = [variable_to_name(all_clusters[i]) for i in range(len(all_clusters))]
all_clusters = combine_clusters(all_clusters, labels)

In [ ]:
plot_bubbles('prod_cluster', 'pay_cluster', all_clusters)
plot_bubbles('prod_cluster', 'skills_cluster', all_clusters)
plot_bubbles('prod_cluster', 'lstandards_cluster', all_clusters)